In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import os

class CSVUploaderApp:
    def __init__(self, master):
        # Use a cleaner theme
        style = ttk.Style()
        style.theme_use('clam')

        self.master = master
        self.master.title("Parent Child File Upload")
        self.master.geometry("450x350")
        self.master.configure(bg="#F5F5F5")

        # Fonts
        heading_font = ("Helvetica", 16, "bold")
        label_font = ("Helvetica", 12)
        button_font = ("Helvetica", 12, "bold")

        # Title
        tk.Label(
            master,
            text="Upload Parent Child Data and Most Recent Invoice",
            font=heading_font,
            bg="#F5F5F5",
            fg="#333"
        ).pack(pady=20)

        # Parent/Child file input
        self.file1_label = tk.Label(master, text="File 1 (Parent Child Data):", font=label_font, bg="#F5F5F5", fg="#333")
        self.file1_label.pack(anchor="w", padx=20)
        self.file1_entry = ttk.Entry(master, width=45)
        self.file1_entry.pack(pady=5, padx=20)
        self.file1_button = ttk.Button(master, text="Browse", command=self.load_file1)
        self.file1_button.pack(pady=5)

        # Invoice file input
        self.file2_label = tk.Label(master, text="File 2 (Invoice):", font=label_font, bg="#F5F5F5", fg="#333")
        self.file2_label.pack(anchor="w", padx=20)
        self.file2_entry = ttk.Entry(master, width=45)
        self.file2_entry.pack(pady=5, padx=20)
        self.file2_button = ttk.Button(master, text="Browse", command=self.load_file2)
        self.file2_button.pack(pady=5)

        # Process button
        self.process_button = ttk.Button(master, text="Process Files", command=self.process_files)
        self.process_button.pack(pady=20)

        # Handle macOS rendering quirks
        self.master.after(100, self.master.update_idletasks)

    def load_file1(self):
        path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
        self.file1_entry.delete(0, tk.END)
        self.file1_entry.insert(0, path)
        self.master.update_idletasks()

    def load_file2(self):
        path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
        self.file2_entry.delete(0, tk.END)
        self.file2_entry.insert(0, path)
        self.master.update_idletasks()

    def process_files(self):
        self.master.update_idletasks()
        path1 = self.file1_entry.get()
        path2 = self.file2_entry.get()

        if not path1 or not path2:
            messagebox.showerror("Error", "Please upload both CSV files.")
            return
        if not path1.endswith('.csv') or not path2.endswith('.csv'):
            messagebox.showerror("Error", "Files must be in CSV format.")
            return

        try:
            os.makedirs('Cache', exist_ok=True)
            os.makedirs('April 1/Output', exist_ok=True)

            cleaned_file = self.remove_duplicates(path2)
            added, not_added, existing = self.load_and_process_files(path1, cleaned_file)

            self.save_modifications(added, not_added, existing)

            messagebox.showinfo("Success", "Files processed successfully!")
        except KeyError as ke:
            messagebox.showerror("Error", f"Missing column: {ke}")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def remove_duplicates(self, file_path):
        self.master.update_idletasks()

        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, encoding='ISO-8859-1')

        df_clean = df[['Account_Number', 'Account_Name']].drop_duplicates(subset='Account_Number')
        df_clean.columns = ['Billing Account Number', 'Billing Account Name']
        output_path = 'Cache/no_duplicates_file.csv'
        df_clean.to_csv(output_path, index=False, encoding='utf-8')

        return output_path

    def load_and_process_files(self, file1_path, file2_path):
        self.master.update_idletasks()

        try:
            df1 = pd.read_csv(file1_path, encoding='utf-8')
        except UnicodeDecodeError:
            df1 = pd.read_csv(file1_path, encoding='ISO-8859-1')

        try:
            df2 = pd.read_csv(file2_path, encoding='utf-8')
        except UnicodeDecodeError:
            df2 = pd.read_csv(file2_path, encoding='ISO-8859-1')

        df1 = df1.iloc[:, :3]

        added, not_added, existing = [], [], []

        for _, row in df2.iterrows():
            acct_num = row['Billing Account Number']
            acct_name = row['Billing Account Name']

            if acct_num in df1['Billing Account Number'].values:
                existing.append(row.to_dict())
                continue

            if acct_name in df1['Billing Account Name'].values:
                idxs = df1.index[df1['Billing Account Name'] == acct_name].tolist()
                if idxs:
                    last_idx = idxs[-1]
                    insert_row = pd.DataFrame([row])
                    df1 = pd.concat([df1.iloc[:last_idx+1], insert_row, df1.iloc[last_idx+1:]], ignore_index=True)
                    df1.at[last_idx+1, 'Parent Name'] = df1.at[last_idx, 'Parent Name']
                    added.append(row.to_dict())
                else:
                    not_added.append(row.to_dict())
            else:
                not_added.append(row.to_dict())

        df1.to_csv('April 1/Output/Updated Parent Child Mapping.csv', index=False, encoding='utf-8')
        return added, not_added, existing

    def save_modifications(self, added, not_added, existing):
        self.master.update_idletasks()

        data = {
            'Items Added': [str(x) for x in added],
            'Items Not Added': [str(x) for x in not_added],
            'Items With Account Number Already Existing': [str(x) for x in existing]
        }

        max_len = max(len(added), len(not_added), len(existing))
        for key in data:
            data[key] += [''] * (max_len - len(data[key]))

        df = pd.DataFrame(data)
        df.to_csv('April 1/Output//Modifications.csv', index=False)

# App entry
if __name__ == "__main__":
    root = tk.Tk()
    app = CSVUploaderApp(root)
    root.mainloop()
